In [1]:
import pandas as pd

#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/03 15:15:47 WARN Utils: Your hostname, LAPTOP-F054CH70 resolves to a loopback address: 127.0.1.1; using 192.168.15.74 instead (on interface eth0)
22/10/03 15:15:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 15:15:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
## read the merchant data
merchant_info = spark.read.options(header = True).csv('../data/curated/merchant_info.csv')
merchant_info = merchant_info.toPandas()

## convert dataset columns into fesible types
merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

In [3]:
list(merchant_info.keys())

['merchant_abn',
 'transaction_count',
 'field',
 'take_rate',
 'revenue_level',
 'total_revenue',
 'mean_consumer_income',
 'fraud_count',
 'main_business_area_popu']

In [18]:
## Manually set the score criteria, could be changed later.
score_criteria = { \
    'transaction_count': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [5]:
## store all numeric features into list for use in both Algorithms
featrue_name = list(merchant_info.keys())
featrue_name.remove('merchant_abn')
featrue_name.remove('field')

In [6]:
score_criteria

{'transaction_count': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'take_rate': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'revenue_level': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4},
 'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'mean_consumer_income': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'fraud_count': {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0},
 'main_business_area_popu': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

## Mark Algorithm

In [7]:
## Function used to assign continuous features into level (a,b,c,d,e) based on current merchant infomation
## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.

def mark_algorithm(merchant_info, feature, score_criteria):

    label_list = []
    score_list = []

    ## in each run of the ranking system, each numeric column is allocated with a level in (a,b,c,d,e)
    ## according to the overall quantile statistic of the feature column we are working on
    ab = merchant_info[feature].quantile(0.8)
    bc = merchant_info[feature].quantile(0.6)
    cd = merchant_info[feature].quantile(0.4)
    de = merchant_info[feature].quantile(0.2)
    
    for i in range(len(merchant_info)):
        if merchant_info.loc[i, feature] >= ab:
            label_list.append('a')
        elif (merchant_info.loc[i, feature] >= bc) & (merchant_info.loc[i, feature] < ab):
            label_list.append('b')
        elif (merchant_info.loc[i, feature] >= cd) & (merchant_info.loc[i, feature] < bc):
            label_list.append('c')
        elif (merchant_info.loc[i, feature] >= de) & (merchant_info.loc[i, feature] < cd):
            label_list.append('d')
        elif merchant_info.loc[i, feature] < de:
            label_list.append('e')

    ## mark each entry of this feature column according to the score_criteria given.
    for j in label_list:
        score_list.append(score_criteria[feature][j])

    return score_list

## Rank Algorithm

In [8]:
## Function used to rank mechant iteratively and then find the top 10 merchants to collaborate.

## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.
## remove_rate (a positive float {0,1}) is the rate to remove merchants classified as poor choice
## top_n (a positive integer) is the number of merchant to be recommended.

## Note: the lower the remove_rate, the higher the accuracy but also the longer the waiting time. (too low may cause overfitting)

def rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n):
    # Stop when we find the asked number of merchants as recommendation
    while (len(merchant_info) > top_n) & (len(merchant_info) * (1-remove_rate) > top_n):
        # In each run, set the score of all merchant to zero as a fresh analysis
        merchant_info['score'] = 0

        # The main part of our rank system to mark and sum scores of all feature columns 
        for feature in feature_name:
            if feature == 'revenue_level':
                for i in range(len(merchant_info)):
                    merchant_info.loc[i, 'score'] += score_criteria[feature][merchant_info.loc[i, feature]]
            else:
                # use of mark algorthm
                merchant_info['score'] += mark_algorithm(merchant_info, feature, score_criteria)

        # sort all merchants by their total score        
        merchant_info = merchant_info.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        # remove a percentage number of merchants having the lowest score (according to remove_rate given)
        merchant_info = merchant_info.drop(merchant_info.tail(int(len(merchant_info) * remove_rate)).index)
        # remove the score column after sorting and dropping merchants. (keep data integrity for next run of this algorithm)
        merchant_info = merchant_info.drop(['score'], axis=1)
        
        # if the stop criteria is not met, keep running to remove merchants that are less likely to be beneficial.
        return rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n)
    
    # Show the top n merchants once stop criteria are met.
    return merchant_info.head(top_n)

In [21]:
## Run Run Run!!!!!!!!!!
top_n_merchant = rank_algorithm(merchant_info, featrue_name, score_criteria, 0.1, 100)

In [22]:
top_n_merchant

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,45629217853,198936,"gift, card, novelty, and souvenir shops",6.98,a,7325106.13,824.636026,43,261909
1,77338620996,10803,"computers, computer peripheral equipment, and ...",3.59,b,5379965.48,837.562418,5,489181
2,32361057556,74770,"gift, card, novelty, and souvenir shops",6.61,a,8218143.78,827.606967,18,273854
3,89726005175,188223,tent and awning shops,6.01,a,7764006.97,824.330699,37,350738
4,64403598239,98928,"music shops - musical instruments, pianos, and...",6.31,a,7726723.97,830.016924,19,225034
...,...,...,...,...,...,...,...,...,...
95,40561597210,148,"music shops - musical instruments, pianos, and...",6.21,a,121594.74,923.068241,0,319102
96,47797405944,4,art dealers and galleries,3.75,b,24196.64,925.750015,1,387326
97,68645868338,6756,motor vehicle supplies and new parts,6.84,a,1263815.37,852.016148,2,93583
98,44780100984,6462,"florists supplies, nursery stock, and flowers",4.86,b,956655.77,834.786660,1,509858
